In [27]:
import os
import time
import urllib.request
import json
import zipfile
from datetime import datetime, timedelta

In [28]:
with open('/Users/tedting/Documents/FactorTradingSystem/data_center/config/full_futures_symbol_list.json') as f:
    futures_symbol_list = json.loads(f.read())

In [30]:
futures_symbol_list

['1000BONKUSDT',
 '1000BTTCUSDT',
 '1000FLOKIUSDT',
 '1000LUNCBUSD',
 '1000LUNCUSDT',
 '1000PEPEUSDC',
 '1000PEPEUSDT',
 '1000RATSUSDT',
 '1000SATSUSDT',
 '1000SHIBBUSD',
 '1000SHIBUSDC',
 '1000SHIBUSDT',
 '1000XECUSDT',
 '1INCHUSDT',
 'AAVEUSDT',
 'ACEUSDT',
 'ACHUSDT',
 'ADABUSD',
 'ADAUSDT',
 'AEVOUSDT',
 'AGIXBUSD',
 'AGIXUSDT',
 'AGLDUSDT',
 'AIUSDT',
 'AKROUSDT',
 'ALGOUSDT',
 'ALICEUSDT',
 'ALPHAUSDT',
 'ALTUSDT',
 'AMBBUSD',
 'AMBUSDT',
 'ANCBUSD',
 'ANCUSDT',
 'ANKRUSDT',
 'ANTUSDT',
 'APEBUSD',
 'APEUSDT',
 'API3USDT',
 'APTBUSD',
 'APTUSDT',
 'ARBUSDT',
 'ARKMUSDT',
 'ARKUSDT',
 'ARPAUSDT',
 'ARUSDT',
 'ASTRUSDT',
 'ATAUSDT',
 'ATOMUSDT',
 'AUCTIONBUSD',
 'AUCTIONUSDT',
 'AUDIOUSDT',
 'AVAXBUSD',
 'AVAXUSDC',
 'AVAXUSDT',
 'AXLUSDT',
 'AXSUSDT',
 'BADGERUSDT',
 'BAKEUSDT',
 'BALUSDT',
 'BANDUSDT',
 'BATUSDT',
 'BCHUSDC',
 'BCHUSDT',
 'BEAMXUSDT',
 'BELUSDT',
 'BICOUSDT',
 'BIGTIMEUSDT',
 'BLUEBIRDUSDT',
 'BLURUSDT',
 'BLZUSDT',
 'BNBBUSD',
 'BNBUSDC',
 'BNBUSDT',
 'BNTUSDT',

In [31]:
import os
import datetime
import urllib.request
import zipfile
import json
from concurrent.futures import ThreadPoolExecutor
import logging

# Setup basic logging
logging.basicConfig(filename='download_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def download_klines_data(symbol, date, data_dir, file_period, bar_interval):
    bar_interval_path = os.path.join(data_dir, bar_interval)
    os.makedirs(bar_interval_path, exist_ok=True)
    
    symbol_path = os.path.join(bar_interval_path, symbol)
    os.makedirs(symbol_path, exist_ok=True)
    
    file_name = f"{symbol}-{bar_interval}-{date}.zip"
    url = f"https://data.binance.vision/data/futures/um/{file_period}/klines/{symbol}/{bar_interval}/{file_name}"
    file_path = os.path.join(symbol_path, file_name)
    
    try:
        urllib.request.urlretrieve(url, file_path)
        logging.info(f"Successfully downloaded {file_path}")

        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(symbol_path)
            logging.info(f"Successfully extracted {file_name} to {symbol_path}")

        os.remove(file_path)
        logging.info(f"Deleted the zip file: {file_path}")
    except Exception as e:
        logging.error(f"Error handling {file_name}: {e}")
        if os.path.exists(file_path):
            os.remove(file_path)
            logging.info(f"Cleanup incomplete download {file_path}")

def download_for_symbol(symbol, start_date, end_date, data_dir, file_period, bar_interval):
    current_date = start_date
    while current_date < end_date:
        download_klines_data(symbol, current_date.strftime('%Y-%m-%d'), data_dir, file_period, bar_interval)
        current_date += datetime.timedelta(days=1)

def main():
    with open('/Users/tedting/Documents/FactorTradingSystem/data_center/config/full_futures_symbol_list.json') as f:
        futures_symbol_list = json.loads(f.read())
    
    data_dir = '/Users/tedting/Documents/FactorTradingSystem/data_Dean/klines'
    start_time = datetime.datetime.strptime('2021-01-01', '%Y-%m-%d')
    end_time = datetime.datetime.now()

    # Use ThreadPoolExecutor to download data in parallel
    with ThreadPoolExecutor(max_workers=4) as executor:
        for symbol in futures_symbol_list:
            executor.submit(download_for_symbol, symbol, start_time, end_time, data_dir, 'daily', '8h')

if __name__ == "__main__":
    main()

KeyboardInterrupt: 